# Mixing ipyvolume with Bokeh
This example shows how the selection from a ipyvolume quiver plot can be controlled with a bokeh scatter plot and it's selection tools.

## Ipyvolume quiver plot
The 3d quiver plot is done using ipyvolume

In [1]:
import ipyvolume
import ipyvolume.pylab as p3
import vaex

We load some data from vaex, but only use the first 10 000 samples for performance reasons of Bokeh.

In [2]:
ds = vaex.example()
N = 10000

We make a quiver plot using ipyvolume's matplotlib's style api.

In [3]:
p3.figure()
quiver = p3.quiver(ds.data.x[:N], ds.data.y[:N], ds.data.z[:N],
              ds.data.vx[:N], ds.data.vy[:N], ds.data.vz[:N],
                  size=1, size_selected=5, color_selected="grey")
p3.xyzlim(-30, 30)
p3.show()

## Bokeh scatter part
The 2d scatter plot is done using Bokeh.

In [4]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import ipyvolume.bokeh
output_notebook()

Loading BokehJS ...

In [5]:
tools = "wheel_zoom,box_zoom,box_select,lasso_select,help,reset,"
p = figure(title="E Lz space", tools=tools, webgl=True, width=500, height=500)
r = p.circle(ds.data.Lz[:N], ds.data.E[:N],color="navy", alpha=0.2)
# A 'trick' from ipyvolume to link the selection (one way traffic atm)
ipyvolume.bokeh.link_data_source_selection_to_widget(r.data_source, quiver, 'selected')
show(p)

Now try doing a selection and see how the above 3d quiver plot reflects this selection.

In [6]:
import ipywidgets

In [7]:
out = ipywidgets.Output()
with out:
    show(p)

In [8]:
ipywidgets.HBox([out, p3.gcc()])

## Embedding in html

A bit of a hack, but it is possible to embed the widget and the bokeh part into a single html file (use at own risk).

In [9]:
from bokeh.resources import CDN
from bokeh.embed import components

script, div = components((p))
template_options = dict(extra_script_head=script + CDN.render_js() + CDN.render_css(),
                        body_pre="<h2>Do selections in 2d (bokeh)<h2>" + div + "<h2>And see the selection in ipyvolume<h2>")
ipyvolume.embed.embed_html("tmp/bokeh.html",
                           [p3.gcc(), ipyvolume.bokeh.wmh], all_states=True,
                           template_options=template_options)